In [1]:
from jira import JIRA
from dotenv import load_dotenv
import getpass
import os

load_dotenv()

JIRA_URL = os.environ.get("JIRA_URL")
EMAIL = os.environ.get("EMAIL")
API_TOKEN = os.environ.get("JIRA_API_TOKEN")

jira = JIRA(
    server=JIRA_URL,
    basic_auth=(EMAIL, API_TOKEN)
)

# Test connection
user = jira.current_user()
print(f"Connected as: {user}")

Connected as: 712020:4da4c4ca-c19f-47de-bbb6-3c4574de0abd


In [2]:
projects = jira.projects()

print(projects)

[<JIRA Project: key='MJ', name='Magic Jira', id='10066'>, <JIRA Project: key='SMP', name='Sample project', id='10000'>]


In [3]:
issues = jira.search_issues(
    'project = MJ ORDER BY created DESC',
)

for issue in issues:
    print(issue.key, issue.fields.summary)

# Jeu de données

In [18]:
import json

with open("../referentiel/moon_match/moon_match_us.json") as f:
    data = json.load(f)

print(data)

[{'epic': 'Immersive and Innovative User Experience', 'user_stories': [{'user_story': 'As a user, I want to seamlessly enter a virtual reality mode from the main interface to freely explore lunar football stadiums and their infrastructure, experiencing a science-fiction inspired lunar environment, so that I can feel fully immersed in a futuristic setting.', 'acceptance_criteria': ["A clearly labeled 'Enter VR Mode' button is present and accessible from the main interface.", 'Upon entering VR mode, the user can freely navigate (walk, look around) within at least one lunar football stadium and its immediate infrastructure (including seating, tunnels, and control rooms).', 'All visuals and interactive elements (such as doors, seats, and control panels) reflect a lunar, science-fiction inspired ambiance, including low gravity effects, futuristic materials, and visible lunar surface through windows or openings.', 'The VR experience maintains a minimum of 60 FPS on supported devices (Oculus 

In [19]:
print(len(data)) # nb d'epic
print(data[0].keys())  # dict_keys(['epic', 'user_stories'])

5
dict_keys(['epic', 'user_stories'])


In [20]:
print(data[0]["epic"])

Immersive and Innovative User Experience


In [21]:
print(len(data[0]["user_stories"]))
print(data[0]["user_stories"][0].keys())

3
dict_keys(['user_story', 'acceptance_criteria', 'independent', 'negotiable', 'valuable', 'estimable', 'small', 'testable'])


In [22]:
print(data[0]["user_stories"][0]["user_story"])

As a user, I want to seamlessly enter a virtual reality mode from the main interface to freely explore lunar football stadiums and their infrastructure, experiencing a science-fiction inspired lunar environment, so that I can feel fully immersed in a futuristic setting.


In [23]:
print(data[0]["user_stories"][0]["acceptance_criteria"])

["A clearly labeled 'Enter VR Mode' button is present and accessible from the main interface.", 'Upon entering VR mode, the user can freely navigate (walk, look around) within at least one lunar football stadium and its immediate infrastructure (including seating, tunnels, and control rooms).', 'All visuals and interactive elements (such as doors, seats, and control panels) reflect a lunar, science-fiction inspired ambiance, including low gravity effects, futuristic materials, and visible lunar surface through windows or openings.', 'The VR experience maintains a minimum of 60 FPS on supported devices (Oculus Quest 2 and above) with no major frame drops or crashes during a continuous 10-minute exploration session.', 'The user can exit VR mode at any time via a clearly labeled option and is returned to the main interface without loss of progress or application instability.']


In [24]:
print(data[0]["user_stories"][0]["independent"])

True


In [25]:
for epic in data:
    print("EPIC:", epic["epic"])
    for us in epic["user_stories"]:
        print("-", us["user_story"])
        for ac in us["acceptance_criteria"]:
            print("  *", ac)

EPIC: Immersive and Innovative User Experience
- As a user, I want to seamlessly enter a virtual reality mode from the main interface to freely explore lunar football stadiums and their infrastructure, experiencing a science-fiction inspired lunar environment, so that I can feel fully immersed in a futuristic setting.
  * A clearly labeled 'Enter VR Mode' button is present and accessible from the main interface.
  * Upon entering VR mode, the user can freely navigate (walk, look around) within at least one lunar football stadium and its immediate infrastructure (including seating, tunnels, and control rooms).
  * All visuals and interactive elements (such as doors, seats, and control panels) reflect a lunar, science-fiction inspired ambiance, including low gravity effects, futuristic materials, and visible lunar surface through windows or openings.
  * The VR experience maintains a minimum of 60 FPS on supported devices (Oculus Quest 2 and above) with no major frame drops or crashes du

# Versement dans Jira

In [40]:
def create_epic(project_key: str, epic_name: str) -> str:
    epic = jira.create_issue(
        project=project_key,
        summary=epic_name,
        issuetype={'name': 'Epic'}
    )

    print(f"Epic créé: {epic.key} - {epic_name}.")

    return epic.key

def create_user_story(project_key: str, user_story_text: str, acceptance_criteria: list, epic_key: str) -> str:
    description = "\n".join([f"* {ac}" for ac in acceptance_criteria])

    story = jira.create_issue(
        project=project_key,
        summary=user_story_text[:255], # limite à 255 caractères
        description=description,
        issuetype={'name': 'Story'},
        parent={'key': epic_key}
    )

    print(f" Story créée: {story.key}.")

    return story.key

def populate_jira(data: list, project_key: str):
    for epic_data in data:
        epic_name = epic_data["epic"]
        epic_key = create_epic(project_key, epic_name)

        for us_data in epic_data["user_stories"]:
            create_user_story(
                project_key=project_key,
                user_story_text=us_data["user_story"],
                acceptance_criteria=us_data["acceptance_criteria"],
                epic_key=epic_key
            )

    print(f"\nVersement terminé.")


In [39]:
populate_jira(data, PROJECT_KEY)

Epic créé: MJ-53 - Immersive and Innovative User Experience.
 Story créée: MJ-54.
 Story créée: MJ-55.
 Story créée: MJ-56.
Epic créé: MJ-57 - Seamless and Personalized Reservation System.
 Story créée: MJ-58.
 Story créée: MJ-59.
 Story créée: MJ-60.
 Story créée: MJ-61.
Epic créé: MJ-62 - Security and High Performance.
 Story créée: MJ-63.
 Story créée: MJ-64.
 Story créée: MJ-65.
 Story créée: MJ-66.
 Story créée: MJ-67.
Epic créé: MJ-68 - Scalability and Future-Proofing.
 Story créée: MJ-69.
 Story créée: MJ-70.
 Story créée: MJ-71.
 Story créée: MJ-72.
Epic créé: MJ-73 - Compatibility and Accessibility.
 Story créée: MJ-74.
 Story créée: MJ-75.
 Story créée: MJ-76.

 Versement terminé.


In [41]:
def delete_all_issues(project_key: str, dry_run: bool = True):

    jql = f'project = {project_key} ORDER BY created DESC'
    issues = jira.search_issues(jql, maxResults=False)  # maxResults=False pour tout récupérer

    print(f"📋 {len(issues)} tickets trouvés dans {project_key}")

    if dry_run:
        print("\nÀ BLANC - Aucune suppression effectuée")
        for issue in issues:
            print(f"  [À BLANC] Serait supprimé: {issue.key} - {issue.fields.summary[:50]}...")
        return

    deleted_count = 0
    errors = []

    for issue in issues:
        try:
            issue_key = issue.key
            issue_summary = issue.fields.summary[:50]
            issue.delete()
            print(f"  Supprimé: {issue_key} - {issue_summary}...")
            deleted_count += 1
        except Exception as e:
            errors.append((issue.key, str(e)))
            print(f"  Erreur sur {issue.key}: {e}")

    print(f"\n{deleted_count} tickets supprimés")
    if errors:
        print(f"{len(errors)} erreurs rencontrées")

In [ ]:
delete_all_issues(PROJECT_KEY, dry_run=False)